In [3]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:13 http://archive.ubuntu.com/ubuntu focal-bac

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(35)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [6]:
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
vine_df = df.select('review_headline', 'review_body', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase')
vine_df.show(20)



+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+
|     review_headline|         review_body|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+
|   LOVE IT. 6 stars!|Bought this last ...|          5|            0|          0|   N|                N|
|       Shipped fast.|These are the bes...|          5|            1|          1|   N|                Y|
|Good idea if it w...|It looks good, th...|          1|            0|          0|   N|                Y|
|          Five Stars|           Great jig|          5|            0|          0|   N|                Y|
|Great quality sil...|I love swimming i...|          5|            0|          1|   N|                N|
|Love everything a...|Love everything a...|          3|            0|          0|   N|                Y|
|          Four Stars|not the best sock...|          4|

In [7]:
from pyspark.sql.functions import desc

# Drop null values from vine_df
vine_drop_df = vine_df.na.drop()

# Sort the vine_drop_df dataframe
vine_drop_df.orderBy(desc("star_rating"), desc("helpful_votes"), desc("total_votes"), desc("review_body"), desc("vine")).show(35)



+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+
|     review_headline|         review_body|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+
| Got Me Past the TSA|While traveling t...|          5|         7379|       7512|   N|                N|
|Rrrrrripppp....ye...|This is a nuts se...|          5|         7166|       7340|   N|                Y|
|Outside runners, ...|I got the Weslo t...|          5|         6246|       6312|   N|                Y|
|   Was skeptical....|I rarely write a ...|          5|         3136|       3208|   N|                Y|
|Don't let the 25 ...|This is a good, f...|          5|         2839|       2996|   N|                N|
|I love this more ...|I was a little le...|          5|         2784|       2876|   N|                N|
|            Victory!|I purchased this ...|          5|

In [8]:
from pyspark.sql.functions import desc

# Select the columns to include in the filtered dataframe and show the first 20 rows
filtered_df = df.select('review_headline', 'review_body','star_rating', 'vine', 'helpful_votes', 'total_votes', 'verified_purchase')
filtered_df.show(20)

# Sort the filtered dataframe in descending order by the star_rating, helpful_votes, and total_votes columns
sorted_df = filtered_df.orderBy(desc("star_rating"))

# Show the first 20 rows of the sorted dataframe
sorted_df.show(20)



#from pyspark.sql.functions import desc
#filtered_df = df.select('star_rating', 'helpful_votes', 'total_votes').show(20)
#filtered_df.orderBy(desc("star_rating"), desc("helpful_votes"), desc("total_votes")).show(20)

+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|     review_headline|         review_body|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|   LOVE IT. 6 stars!|Bought this last ...|          5|   N|            0|          0|                N|
|       Shipped fast.|These are the bes...|          5|   N|            1|          1|                Y|
|Good idea if it w...|It looks good, th...|          1|   N|            0|          0|                Y|
|          Five Stars|           Great jig|          5|   N|            0|          0|                Y|
|Great quality sil...|I love swimming i...|          5|   N|            0|          1|                N|
|Love everything a...|Love everything a...|          3|   N|            0|          0|                Y|
|          Four Stars|not the best sock...|          4|

In [9]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc


equated_df = sorted_df.filter(col("helpful_votes") / col("total_votes") >= 0.5)
equated_df.show(20)

organized_df = equated_df.orderBy(desc("star_rating"), desc("helpful_votes"), desc("total_votes"))
organized_df.show(20)



# Sort the filtered dataframe in descending order by the total_votes and helpful_votes columns
#sorted_review_df = filtered_review_df.orderBy(col('total_votes').desc(), col('helpful_votes').desc())

# Show the first 20 rows of the sorted dataframe
#sorted_review_df.show(20)






+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|     review_headline|         review_body|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|My impression so ...|Black stretchy ma...|          5|   N|           11|         11|                Y|
|               Happy|Nice shirt to my ...|          5|   N|            1|          1|                Y|
|Good weight and f...|Good weight and f...|          5|   N|            1|          1|                Y|
|can't wait to use it|Unfortunately I h...|          5|   N|            1|          1|                Y|
| Adequate & Reliable|I'm leaving this ...|          5|   N|           11|         11|                Y|
|this mat is a pre...|If you have a ter...|          5|   N|            2|          2|                Y|
|          Five Stars|Works and fits pe...|          5|

In [10]:
from pyspark.sql.functions import col
vine_y_df = sorted_df.filter(col("vine") == 'Y')
vine_y_df.show()

vine_n_df = sorted_df.filter(col("vine") == 'N')
vine_n_df.show()

#vine_y_df = vine_df.select(col("vine"))
#vine_y_df = vine_df.filter
#vine_y_df
#reviews_df = vine_df.filter
#vine_df.show(20)


+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|     review_headline|         review_body|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------------+--------------------+-----------+----+-------------+-----------+-----------------+
|Terrific Practice...|This is a very ni...|          5|   Y|            1|          1|                N|
|Easy to use and r...|It is latex free ...|          5|   Y|            1|          1|                N|
|Love the remote a...|This is my second...|          5|   Y|            1|          1|                N|
|Bit of Flavor. . ...|This is a great “...|          5|   Y|            0|          0|                N|
|WONDERFUL MASSAGE...|You get exactly w...|          5|   Y|            0|          1|                N|
|  Sturdy, well-built|Very sturdy bench...|          5|   Y|            1|          1|                N|
|Subbed in a Bigge...|I have been looki...|          5|

In [31]:
from pyspark.sql.functions import col, count, sum

# Count the total number of reviews for each type
count_reviews_df = vine_n_df.groupBy("vine").agg(count("*").alias("total_reviews"))

# Count the number of 5-star reviews for each type
five_star_reviews_df = vine_n_df.filter(col("star_rating") == 5).groupBy("vine").agg(count("*").alias("five_star_reviews"))

# Join the two DataFrames on the "vine" column
joined_df = five_star_reviews_df.join(count_reviews_df, "vine")

# Calculate the percentage of 5-star reviews for each type
percentage_df = joined_df.withColumn("percentage_five_star", (col("five_star_reviews") / col("total_reviews")) * 100)

# Show the results
percentage_df.show()


+----+-----------------+-------------+--------------------+
|vine|five_star_reviews|total_reviews|percentage_five_star|
+----+-----------------+-------------+--------------------+
|   N|          3035639|      4839483|   62.72651438180483|
+----+-----------------+-------------+--------------------+



In [32]:
# Count the total number of reviews for each type
count_reviews_df = vine_y_df.groupBy("vine").agg(count("*").alias("total_reviews"))

# Count the number of 5-star reviews for each type
five_star_reviews_df = vine_y_df.filter(col("star_rating") == 5).groupBy("vine").agg(count("*").alias("five_star_reviews"))

# Join the two DataFrames on the "vine" column
joined_df = five_star_reviews_df.join(count_reviews_df, "vine")

# Calculate the percentage of 5-star reviews for each type
percentage_df = joined_df.withColumn("percentage_five_star", (col("five_star_reviews") / col("total_reviews")) * 100)

# Show the results
percentage_df.show()


+----+-----------------+-------------+--------------------+
|vine|five_star_reviews|total_reviews|percentage_five_star|
+----+-----------------+-------------+--------------------+
|   Y|             4481|        10080|  44.454365079365076|
+----+-----------------+-------------+--------------------+



In [33]:
# Count the total number of reviews for each type
count_reviews_df = vine_y_df.groupBy("verified_purchase").agg(count("*").alias("total_reviews"))

# Count the number of 5-star reviews for each type
five_star_reviews_df = vine_y_df.filter(col("star_rating") == 5).groupBy("verified_purchase").agg(count("*").alias("five_star_reviews"))

# Join the two DataFrames on the "vine" column
joined_df = five_star_reviews_df.join(count_reviews_df, "verified_purchase")

# Calculate the percentage of 5-star reviews for each type
percentage_df = joined_df.withColumn("percentage_five_star", (col("five_star_reviews") / col("total_reviews")) * 100)

# Show the results
percentage_df.show()

+-----------------+-----------------+-------------+--------------------+
|verified_purchase|five_star_reviews|total_reviews|percentage_five_star|
+-----------------+-----------------+-------------+--------------------+
|                Y|                9|           11|   81.81818181818183|
|                N|             4472|        10069|   44.41354652895024|
+-----------------+-----------------+-------------+--------------------+

